In [ ]:
from __future__ import print_function

import pandas as pd
import numpy as np

from depmapomics import loading, tracker, cn
from depmapomics import mutations as omics_mut
from depmapomics import terra as myterra
from genepy import terra
from genepy import mutations as mut
from genepy.utils import helper as h
from genepy.google import gcp
from gsheets import Sheets
from taigapy import TaigaClient
import dalmatian as dm
from genepy.google.google_sheet import dfToSheet

from IPython.display import Image,display
from bokeh.plotting import *

%load_ext autoreload
%autoreload 2
%load_ext rpy2.ipython
tc = TaigaClient()
output_notebook()

my_id = '~/.client_secret.json'
mystorage_id = "~/.storage.json"
# do the first steps of https://medium.com/craftsmenltd/from-csv-to-google-sheet-using-python-ef097cb014f9
creds = '../.credentials.json'

sheets = Sheets.from_files(my_id, mystorage_id)
replace = {'T': 'Tumor', 'N': 'Normal', 'm': 'Unknown', 'L': 'Unknown'}

In [ ]:
ccle_fingerprint_lod_matrix = tc.get(name='ccle-bam-fingerprinting-db5d', version=1, file='ccle_fingerprint_lod_matrix')

In [ ]:
tc.update_dataset(changes_description="new "+samplesetname+" release! (removed misslabellings, see changelog)",
                  dataset_permaname='ccle-bam-fingerprinting-db5d',
                  add_all_existing_files=True,
                  upload_files=[
                    {
                        "path": "temp/segments_allWES_latest_"+samplesetname+".csv",
                        "name": "segments_allWES_latest_"+samplesetname,
                        "format": "TableCSV",
                        "encoding": "utf-8"
                    },
                      'ccle-bam-fingerprinting-db5d',
                 )

In [ ]:
ccle_refsamples = sheets.get(refsheet_url).sheets[0].to_frame(index_col=0)

In [ ]:
refsheet_url = "https://docs.google.com/spreadsheets/d/1Pgb5fIClGnErEqzxpU7qqX6ULpGTDjvzWwDN8XUJKIY"

In [ ]:
ccle_refsamples = ccle_refsamples[~ccle_refsamples.internal_bam_filepath.isna()]

In [ ]:
ccle_refsamples = ccle_refsamples[ccle_refsamples.internal_bam_filepath.str.contains('CDS-')]

In [ ]:
a = ccle_refsamples[ccle_refsamples.index!='CDS-' + ccle_refsamples.internal_bam_filepath.str.split('CDS-').str[1].str.split('.').str[0]]

In [ ]:
a

In [ ]:
ccle_fingerprint_lod_matrix = ccle_fingerprint_lod_matrix.rename(columns=a).rename(index=a)

In [ ]:
ccle_fingerprint_lod_matrix.loc[i,i]

In [ ]:
for i in ccle_fingerprint_lod_matrix.index:
    if ccle_fingerprint_lod_matrix.loc[i,i] < -5:
        print(i)

In [ ]:
ccle_fingerprint_lod_matrix.to_csv('ccle_fingerprint_lod_matrix.csv')

In [ ]:
ls 

In [ ]:
a

In [ ]:
a = {"CDS-"+v[0].split('CDS-')[1].split('.')[0]: i for i,v in a[['internal_bai_filepath']].iterrows()}

In [ ]:
.iloc[0].values

In [ ]:
ccle_refsamples.loc[a.index.tolist()][['internal_bam_filepath','internal_bai_filepath']].shape

In [ ]:
dfToSheet(ccle_refsamples, "ccle sample tracker", creds)

In [ ]:
a.index.tolist()

In [ ]:
a['internal_bam_filepath']

In [ ]:
ccle_refsamples.loc[a.index.tolist(), ['internal_bam_filepath','internal_bai_filepath']] = a[['internal_bam_filepath','internal_bai_filepath']].values

In [ ]:
'gs://cclebams/rnasq_hg38/'+CDS-1M7qNP+'.Aligned.sortedByCoord.out.bam.bai'